# Handout 04
#### Sara Díaz del Ser

In [2]:
import numpy as np
from collections import Counter
from itertools import combinations
import pandas as pd
from termcolor import colored
import tabulate

### Ex. 1 _(0 pts)_  Manually determining scoring matrices
To test your understanding, consider this small example of a blocks alignment database consisting of
5 sequences of length 12 each. From these to numbers one can determinate the denominators in eqs.
(2) and (3). What is $∑_{i} f_{i}$ and $∑_{{i,j}} f_{ij}$ ?
```
TSVKTYAKFVTH
TSVKTYAKFSTH
TSVKTYAKFVTH
LSVKKYPKYVVQ
SSVKKYPKYSVL
```
Count the frequencies $f_{a}$ for all amino acids in the alignment and $f_{ab}$ for all amino acid pairs occurring
in the same column of the alignment. (For the pairs, do not consider the order of the amino acids: do
not distinguish between VS and SV, for example). From these values, calculate the relative frequencies
$p_{a}$ for each occurring amino acid and $p_{ab}$ for each occurring amino acid pair. Finally, calculate the
expected probability and the score for each amino acid pair. Fill your results into the given tables.

In [3]:
# sequences
seq1 = 'TSVKTYAKFVTH'
seq2 = 'TSVKTYAKFSTH'
seq3 = 'TSVKTYAKFVTH'
seq4 = 'LSVKKYPKYVVQ'
seq5 = 'SSVKKYPKYSVL'

In [4]:
# fa
def fa(seq:str) -> dict:
	"""Return frequency of all amino acids"""
	return Counter(seq)

In [5]:
# fab
def pair_freq(seq1,seq2):
	"""Add pair frequencies from two sequences into dictionary"""
	all_pairs = []
	for i in range(len(seq1)):
		# Append pair (ordered) to list:
		key = sorted([seq1[i],seq2[i]])
		all_pairs.append("".join(key))
	return all_pairs

def all_pair_freq(seq_list:list) -> dict:
	"""Get pair frequencies from list of sequences"""
	all_pairs = []
	for seq1,seq2 in combinations(seq_list,2):
		all_pairs.extend(pair_freq(seq1,seq2))
	return Counter(all_pairs)


In [6]:
# pa (relative frequency for aa)
def rel_freq(freq:dict) -> dict:
	"""Determine relative frequency for each aminoacid"""
	return { key : (val/sum(freq.values())) for key, val in freq.items() }

In [7]:
def expected_prob(freq:dict) -> dict:
	"""Determine the expected probability of each aminoacid"""
	expected = {}
	# Get all possible pairs
	possible_pairs = ["".join(sorted([aa1,aa2])) for aa1,aa2 in combinations(freq.keys(),2) ]
	# add pairs of the same aa
	possible_pairs.extend([aa+aa for aa in freq.keys() ])
	for pair in possible_pairs:
		# eaa = pa * pa
		if pair[0]==pair[1]:
			prob = freq.get(pair[0])**2
		# eab = pa * pb + pb * pa = 2 * pa * pb
		else:
			prob = 2*freq.get(pair[0])*freq.get(pair[1])
		expected.update({pair: prob})
	return expected

In [8]:
def pair_score(p_freq_rel, p_expect) -> dict:
	"""Calculate the score for each aminoacid pair"""
	scores = {}
	for pair in p_freq_rel.keys():
		# sab = 2 * log2(pab/eab)
		score = 2*np.log2(p_freq_rel.get(pair,0)/p_expect.get(pair,0))
		scores.update({pair : round(score,0)})
	return scores

In [9]:
freq = fa(seq1+seq2+seq3+seq4+seq5)

In [10]:
p_freq = all_pair_freq([seq1, seq2,seq3,seq4,seq5])

In [11]:
freq_rel = rel_freq(freq)

In [12]:
p_freq_rel = rel_freq(p_freq)

In [13]:
p_exp = expected_prob(freq)

In [14]:
p_score = pair_score(p_freq_rel, p_exp)

In [15]:
# Fill the tables
single_df = pd.DataFrame([freq, freq_rel], ['Fa','Pa']).round(decimals =2)
single_df

,T,S,V,K,Y,A,F,H,L,P,Q
Fa,9.00,8.00,10.00,12.0,7.00,3.00,3.00,3.00,2.00,2.00,1.00
Pa,0.15,0.13,0.17,0.2,0.12,0.05,0.05,0.05,0.03,0.03,0.02


In [16]:
pair_df = pd.DataFrame([p_freq, p_freq_rel], ['Fab','Pab']).round(decimals =2)
pair_df

,TT,SS,VV,KK,YY,AA,FF,SV,HH,LT,KT,AP,FY,TV,HQ,ST,HL,LS,PP,LQ
Fab,9.00,11.00,14.00,21.00,11.00,3.00,3.00,6.00,3.00,3.00,6.00,6.00,6.00,6.00,3.00,3.00,3.00,1.00,1.00,1.00
Pab,0.08,0.09,0.12,0.18,0.09,0.02,0.02,0.05,0.02,0.02,0.05,0.05,0.05,0.05,0.02,0.02,0.02,0.01,0.01,0.01


### Ex.2 _(7 pts)_ Calculating scoring matrices

Write a program that computes a scoring matrix score from a given block alignment database. You
should be able to call your problem like ```python blosum.py alignment.dat blosum_matrix.out``` from
the command line. The input file alignment.dat (found in the course folder) contains a small alignment
"database". It contains a number of aligned sequences of equal length with no gaps with one sequence
per line.Your program should compute the substitution matrix from this alignment. The output file
```blosum_matrix.out``` should contain an output of the matrix like in the example of the BLOSUM matrix
given above. (The original BLOSUM62 matrix is provided as blosum62.txt in the group folder. Are
the scores of the matrix you have calculated for ```alignment.dat``` similar?)

Your program will need to.
(a) read in the alignment data in an appropriate data structure
(b) determine the log-odds scores for each possible alignment of amino acids,
(c) produce a (nicely formatted) output of the resulting scoring matrix.


In [17]:
# (a) read in the alignment data in an appropriate data structure
def read_file(file:str) -> list:
	"""Read file into list of lines"""
	with open (file, 'r') as f:
		return f.readlines()

file = './alignment.dat'
seqs = read_file(file)

In [18]:
def pre_processing(seqs:list) -> list:
	"""Pre-process alignment data:
	- Add padding at the end of shorter strings to make sure they're the same size
	- Remove newlines
	"""
	max_length = max([len(each) for each in seqs ])
	seqs = [ seq.ljust(max_length).replace('\n','') for seq in seqs ]
	return seqs

seqs = pre_processing(seqs)

In [19]:
# (b) determine the log-odds scores for each possible alignment of amino acids
def calc_score(seqs:list) -> dict:
	"""Determine score from list of pre-processed aligned sequences"""
	freq = fa("".join(seqs))
	p_freq = all_pair_freq(seqs)

	freq_rel = rel_freq(freq)
	p_freq_rel = rel_freq(p_freq)

	p_exp = expected_prob(freq)
	p_score = pair_score(p_freq_rel, p_exp)

	return p_score

score = calc_score(seqs)

In [20]:
# (c) produce a (nicely formatted) output of the resulting scoring matrix.
def aminoacids(score:dict) -> dict:
	"""Asigns an index to each aminoacid"""
	list_aa = []
	[list_aa.extend([key[0],key[1]]) for key in score.keys()]
	list_aa = set(list_aa)
	return { key: i for i,key in enumerate(list_aa)}

In [21]:
def calc_score_matrix(score:dict) -> np.ndarray:
	"""Generate a score matrix from a score dictionary"""
	# Create matrix of 0 of size (20,20)
	matrix = np.zeros((20,20),dtype=int)

	# Get index for each aa in matrix
	aa_index = aminoacids(score)

	# Iterate through score dictionary and input values into matrix
	for key, val in score.items():
		matrix[aa_index[key[0]],aa_index[key[1]]] = val

	return matrix

score_matrix = calc_score_matrix(score)

In [22]:
def score_table(score_matrix:np.ndarray,score, file) -> pd.DataFrame:
	"""Turn a numpy score matrix into a dataframe"""
	aa_index = aminoacids(score)
	columns = list(aa_index.keys())

	df = pd.DataFrame(score_matrix, columns=columns)
	df = df.rename(dict(enumerate(columns)))

	with open(file, 'w') as f:
		print(df.to_string(), file=f)
		print(colored(f'Score matrix saved to "{file}"'), 'green')
	return df

In [23]:
score_table(score_matrix, score, 'score_matrix_test.txt')

# TODO start off the frequencies as 1, to avoid problems with 0

Score matrix saved to "score_matrix_test.txt" green


,T,Y,N,I,R,C,D,H,F,P,W,A,E,M,L,G,Q,V,K,S
T,-41,-49,0,0,0,0,0,0,0,0,-51,0,0,0,0,0,0,-49,0,0
Y,0,-38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
N,-47,-50,-39,0,-48,0,0,0,0,-50,-53,0,0,0,0,0,-48,-48,0,-47
I,-48,-49,-48,-41,-50,0,0,0,0,-50,-52,0,0,-47,-46,0,-47,-45,-51,-49
R,-48,-48,0,0,-40,0,0,0,0,0,-51,0,0,0,0,0,0,-50,0,-50
C,-47,-47,-50,-48,-47,-39,-50,0,-46,-48,-50,0,-49,-46,-48,-48,-46,-48,-50,-48
D,-52,-51,-46,-49,-51,0,-41,-50,-50,-52,-55,0,-45,-52,-50,-49,-53,-49,-50,-50
H,-50,-46,-48,-46,-46,0,0,-39,0,-49,-52,0,0,-47,-49,0,-44,-50,-51,-48
F,-50,-44,-51,-48,-49,0,0,-51,-40,-52,-47,0,0,-47,-46,-53,-50,-50,-57,-52
P,-49,-50,0,0,-52,0,0,0,0,-40,-52,0,0,0,0,0,-51,-52,0,-47


#### Run functions as script

In [24]:
%run diazdelser-scoring.py 'alignment.dat' 'score_matrix.txt'

Score matrix saved to "score_matrix.txt" green


### Ex.3 _(13 pts)_ Global/local alignment

#### (a) _(5 pts)_ Implement the Needleman-Wunsch algorithm:
Write a program called needle.py that takes 4 parameters
and should be called like ```python needle.py -8 blosum.txt sequence1.fasta sequence2.fasta```, where -8 is the gap
penalty w, ```blosum.txt``` is a text file containing the scoring matrix and ```sequence1.fasta``` and
```sequence2.fasta``` are two amino acid sequences in Fasta format.

The program should output the score of the alignment and the aligned sequences, i.e. the sequences
plus the gaps (represented by the symbol -).

• Note that you will need to write a function that is able to read in a scoring matrix from a
text file and store it in a suitable data structure.

• Test your program using the scoring matrix blosum62.dat and a gap penalty of -5 on the
sequences:
THRQATWQPPLERMANGRQVE and RAYMQNDLVKVRYYACHT

In [25]:
seqA = 'THRQATWQPPLERMANGRQVE'
seqB = 'RAYMQNDLVKVRYYACHT'

In [26]:
def score_matrix_to_df(lines:list) -> (str, list):
	"""Read list of lines into dataframe"""
	lines = [line.strip() for line in lines]
	# Turn lines into list of scores
	lines =[ [ letter for letter in line.split() if letter !=' '] for line in lines ]
	# Separate header from rest of the strings
	header = lines[0]
	# Remove first item from lists (it's the aa)
	lines = [line[1:] for line in lines[1:]]
	# Generate pandas dataframe, remove emtpy rows
	df = pd.DataFrame(lines, columns=header).dropna(how='all')
	# Add aa symbols as row headers
	df = df.rename(dict(enumerate(header)))
	return df

In [177]:
# Open score matrix file
text = read_file('./blosum62.dat')
# Read into df
score = score_matrix_to_df(text)

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,-1,-1,-1,-2,-1,1,0,-3,-2,0
R,-1,5,0,-2,-3,1,0,-2,0,-3,-2,2,-1,-3,-2,-1,-1,-3,-2,-3
N,-2,0,6,1,-3,0,0,0,1,-3,-3,0,-2,-3,-2,1,0,-4,-2,-3
D,-2,-2,1,6,-3,0,2,-1,-1,-3,-4,-1,-3,-3,-1,0,-1,-4,-3,-3
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,-1,-3,-1,-2,-3,-1,-1,-2,-2,-1
Q,-1,1,0,0,-3,5,2,-2,0,-3,-2,1,0,-3,-1,0,-1,-2,-1,-2
E,-1,0,0,2,-4,2,5,-2,0,-3,-3,1,-2,-3,-1,0,-1,-3,-2,-2
G,0,-2,0,-1,-3,-2,-2,6,-2,-4,-4,-2,-3,-3,-2,0,-2,-2,-3,-3
H,-2,0,1,-1,-3,0,0,-2,8,-3,-3,-1,-2,-1,-2,-1,-2,-2,2,-3
I,-1,-3,-3,-3,-1,-3,-3,-4,-3,4,2,-3,1,0,-3,-2,-1,-3,-1,3


In [158]:
def create_alignment_matrix(seqA:str, seqB:str,w, score) -> (np.ndarray,np.ndarray):
	"""Create a Neeedleman-Wunsch alignment matrix"""
	#Initialize: M(0; 0) = 0
	matrix = np.zeros((len(seqA),len(seqB)),dtype=int)
	traceback = np.zeros((len(seqA),len(seqB)),dtype=tuple)
	# first row M(0; j) = jw for j = 1 ... m,
	for i in range(len(seqA)-1):
		matrix[i,0] = i*w
	# first column M(i; 0) = iw for i = 1 ... n
	for j in range(len(seqB)-1):
		matrix[0,j] = j*w
		pass
	# Fill Rest of the matrix
	for i in range(len(seqA)-1):
		for j in range(len(seqB)-1):
			options = {
				matrix[i-1,j-1] + int(score.loc[seqA[i], seqB[j]]) : (i-1,j-1),
				matrix[i-1,j] + w : (i-1,j),
				matrix[i,j-1] : (i,j-1)
			}
			best_option = max(options.keys())
			matrix[i,j] = best_option
			traceback[i,j] = options.get(best_option)
	return matrix, traceback


In [159]:
def traceback(i,j,traceback_matrix, seqA,seqB,alignement):
	"""Check traceback"""
	if traceback_matrix[i,j] == (i-1,j-1):
		alignement.append((seqA[j],seqB[i]))
		traceback(i-1,j-1,traceback_matrix, seqA,seqB,alignement)
	elif traceback_matrix[i,j] == (i-1,j):
		alignement.append((seqA[j],'-'))
		traceback(i-1,j,traceback_matrix, seqA,seqB,alignement)
	elif traceback_matrix[i,j] ==(i,j-1):
		alignement.append(('-',seqB[i]))
		traceback(i,j-1,traceback_matrix, seqA,seqB,alignement)
	return alignement


In [180]:
def print_alignment(alignement:list):
	"""Print sequence alignment"""
	x,y=zip(*alignement)
	print("".join(x))
	print("".join(['|' if x == y else ' ' for x,y in zip(x,y) ]))
	print("".join(y))

In [230]:
def total_score(alignement:list, score_matrix:pd.DataFrame) -> int:
	x,y=zip(*alignement)
	scores = [ int(score_matrix.loc[x,y]) for x,y in zip(x,y) if (x != '-') and (y!= '-') ]
	return sum(scores)

In [231]:
def Needleman_Wunsch(seqA:str, seqB:str, score_matrix:pd.DataFrame, w:int) -> int:
	"""Runs Neeedleman-Wunsch Algorithm"""
	# Construction of alignment matrix
	matrix, traceback_matrix = create_alignment_matrix(seqB,seqA, w,score_matrix)

	# Construct alignment via traceback matrix
	n = len(traceback_matrix)
	m = len(traceback_matrix[0])
	alignement = []
	alignement = traceback(n-2,m-2,traceback_matrix, seqA,seqB,alignement)

	#print_alignment(alignement)
	return total_score(alignement, score_matrix)

In [232]:
# Try Needleman_Wunsch algorithm
Needleman_Wunsch(seqA=seqA, seqB=seqB, score_matrix=score, w=-3)



27

#### (b) _(2 pts)_ Formatted output.
Modifiy the output of the alignment so that sequence 1 appears in
the first line and sequence 2 in the third line while the second line contains symbols reflecting
the similarity between aligned symbols:
 - for identical residues,
 - : for conservative substitutions (positive score in scoring matrix)
 - and spaces otherwise.


In [243]:
def print_alignment(alignement:list,score_matrix:pd.DataFrame):
	"""Print sequence alignment"""
	X,Y=zip(*alignement)
	score_symbol = []
	for x,y in zip(X,Y):
		if x == y:
			score_symbol.append('|')
		elif (x=='-') or (y=='-'):
			score_symbol.append(" ")
		elif int(score_matrix.loc[x,y]) > 0:
			score_symbol.append(':')
		else:
			score_symbol.append(" ")

	print("".join(X))
	print("".join(score_symbol))
	print("".join(Y))

In [244]:
def Needleman_Wunsch(seqA:str, seqB:str, score_matrix:pd.DataFrame, w:int) -> int:
	"""Runs Neeedleman-Wunsch Algorithm"""
	# Construction of alignment matrix
	matrix, traceback_matrix = create_alignment_matrix(seqB,seqA, w,score_matrix)

	# Construct alignment via traceback matrix
	n = len(traceback_matrix)
	m = len(traceback_matrix[0])
	alignement = []
	alignement = traceback(n-2,m-2,traceback_matrix, seqA,seqB,alignement)

	print_alignment(alignement,score_matrix)
	return total_score(alignement, score_matrix)


In [245]:
# Try Needleman_Wunsch algorithm
s = Needleman_Wunsch(seqA=seqA, seqB=seqB, score_matrix=score, w=-3)


----NAAMMREELLPPQWW-A-R
    : |  | : |  | : | |
HHHHH-A-YR-K-LDNQ-YAARR


# TODO
Break the output into multiple lines of about 80 characters so that alignment of long sequences
is displayed properly.

#### (c) _(2 pts)_ Testing.
Perform all pairwise sequence alignments between sequences
```RNAS1_horse.fasta```, ```RNAS1_minke-whale.fasta```, and ```RNAS1_red-kangaroo.fasta``` using ```blosum50.dat```
and a gap penalty of -8. What can you conclude about the pairwise relationships?

#### (d) _(2 pt)_ Smith-Waterman.
Implement the Smith-Waterman algorithm by modifying the code from
```needle.py```. In addition to the alignment score you should output the sequence similarity and the
sequence identity of the two aligned sequences in percent. Sequence identity is the percentage
of matching residues relative to the length of the aligned sequences including gaps, sequence
similarity is the percentage of matched similar residues (i.e., those with a positive value in the
scoring matrix) relative to the length of the aligned sequences including gaps.

#### (e) _(2 pt)_ Testing.
Run a global and a local sequence alignment on the sequences ```halodurans.fasta```
and ```lentus.fasta``` using ```blosum62.txt``` and a gap penalty of -8. Which is more appropriate?